# Contoh Penggunaan Alat AutoGen


## Impor Paket yang Dibutuhkan


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Membuat Client

Dalam contoh ini, kita akan menggunakan [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) untuk mengakses LLM.

`model` didefinisikan sebagai `gpt-4o-mini`. Cobalah mengganti model ke model lain yang tersedia di marketplace GitHub Models untuk melihat hasil yang berbeda.

Sebagai uji coba cepat, kita akan menjalankan prompt sederhana - `Apa ibu kota Prancis`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Mendefinisikan Fungsi

Dalam contoh ini, kita akan memberikan agen akses ke sebuah alat yang merupakan fungsi dengan daftar destinasi liburan yang tersedia beserta ketersediaannya.

Anda bisa membayangkan bahwa ini adalah skenario di mana seorang agen perjalanan mungkin memiliki akses ke database perjalanan, misalnya.

Saat Anda mengikuti contoh ini, jangan ragu untuk mencoba mendefinisikan fungsi dan alat baru yang dapat dipanggil oleh agen.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Mendefinisikan Function Tool
Agar agen dapat menggunakan `vacation_destinations` sebagai `FunctionTool`, kita perlu mendefinisikannya sebagai salah satu alat.

Kita juga akan memberikan deskripsi alat tersebut yang berguna bagi agen untuk mengidentifikasi kegunaan alat tersebut dalam kaitannya dengan tugas yang diminta oleh pengguna.


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Mendefinisikan agen

Sekarang kita dapat membuat agen menggunakan kode di bawah ini. Kita mendefinisikan `system_message` untuk memberikan instruksi kepada agen tentang cara membantu pengguna menemukan destinasi liburan.

Kita juga mengatur parameter `reflect_on_tool_use` ke true. Ini memungkinkan LLM untuk mengambil respons dari pemanggilan alat dan mengirimkan respons tersebut menggunakan bahasa alami.

Anda dapat mengatur parameter ini ke false untuk melihat perbedaannya.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Menjalankan Agen

Sekarang kita dapat menjalankan agen dengan pesan awal dari pengguna yang meminta untuk melakukan perjalanan ke Tokyo.

Anda dapat mengganti tujuan kota ini untuk melihat bagaimana agen merespons ketersediaan kota tersebut.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan penerjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berusaha untuk memberikan hasil yang akurat, harap diingat bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang otoritatif. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemahan profesional oleh manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
